# 2. Data Cleaning and Feature Engineering

**Author:** Carlos Gómez Gómez

## Objective
This notebook takes the raw, scraped data from the `01_Web_Scraping` notebook, performs a series of cleaning and feature engineering steps, and saves a final, analysis-ready dataset.

The main steps are:
1.  **Load and Combine Data:** Merge all individual CSV files into a single DataFrame.
2.  **Initial Cleaning:** Handle basic issues like duplicate columns and drop rows with critical missing values.
3.  **Feature Engineering:** Create new, useful columns from existing text-based data (e.g., extracting `floor_number` from a text description).
4.  **Handle Missing Values:** Impute remaining nulls using appropriate strategies (median for numerical, default value for categorical).
5.  **Final Encoding:** Convert all categorical and boolean features into a numerical format suitable for machine learning.
6.  **Save Cleaned Data:** Export the final, cleaned DataFrame to a new CSV file.

In [10]:
# --- 1. Library Imports ---
import pandas as pd
import numpy as np
import re

## 2. Load and Combine Raw Data

First, we load the 10 separate CSV files that were generated by the scraper for each district of Barcelona. We then concatenate them into a single raw DataFrame.

In [11]:
# --- Load individual district data ---
try:
    ciutat_vella_df = pd.read_csv('../data/data_barcelona_ciutat-vella.csv')
    eixample_df = pd.read_csv('../data/data_barcelona_eixample.csv')
    gracia_df = pd.read_csv('../data/data_barcelona_gracia.csv')
    horta_guinardo_df = pd.read_csv('../data/data_barcelona_horta-guinardo.csv')
    les_corts_df = pd.read_csv('../data/data_barcelona_les-corts.csv')
    nou_barris_df = pd.read_csv('../data/data_barcelona_nou-barris.csv')
    sant_andreu_df = pd.read_csv('../data/data_barcelona_sant-andreu.csv')
    sants_montjuic_df = pd.read_csv('../data/data_barcelona_sants-montjuic.csv')
    sant_marti_df = pd.read_csv('../data/data_barcelona_sant-marti.csv')
    sarria_gervasi_df = pd.read_csv('../data/data_barcelona_sarria-sant-gervasi.csv')

    # --- Combine into a single DataFrame ---
    df_list = [
        ciutat_vella_df, eixample_df, gracia_df, horta_guinardo_df, 
        les_corts_df, nou_barris_df, sant_andreu_df, sants_montjuic_df,
        sant_marti_df, sarria_gervasi_df
    ]
    
    raw_df = pd.concat(df_list, ignore_index=True)

    # Clean up any 'Unnamed' columns that may have been created during saving
    raw_df = raw_df.loc[:, ~raw_df.columns.str.contains('^Unnamed')]

    print("Data combined successfully.")
    raw_df.info()
    
    # --- Save the combined raw dataset to a new CSV file ---
    combined_filepath = '../data/barcelona_properties_raw_combined.csv'
    raw_df.to_csv(combined_filepath, index=False)
    print(f"\nCombined raw dataset saved to '{combined_filepath}'")
    
except FileNotFoundError as e:
    print(f"Error: Could not find a CSV file. Make sure the scraped data exists in the '../data/' directory.")
    print(e)

Data combined successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11830 entries, 0 to 11829
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              11830 non-null  int64  
 1   title           11807 non-null  object 
 2   location        11807 non-null  object 
 3   city            11808 non-null  object 
 4   price           11807 non-null  float64
 5   original_price  11807 non-null  object 
 6   discount        11830 non-null  int64  
 7   m2              11807 non-null  float64
 8   hab             11546 non-null  float64
 9   floor           9920 non-null   object 
 10  air             11830 non-null  bool   
 11  terrace         11830 non-null  bool   
 12  storage         11830 non-null  bool   
 13  elevator        11830 non-null  bool   
 14  garage          11830 non-null  bool   
 15  outside         11830 non-null  bool   
 16  wardrobes       11830 non-null  bool   
 17  poo

## 3. Initial Cleaning and Preprocessing

In this step, we perform some basic data cleaning tasks:
- Drop columns that are not useful for the analysis or have too many missing values (`energy`, `co2`).
- Remove rows that are missing essential information like `price` or `m2`, as they cannot be used for modeling.
- Clean the `original_price` column to make it a numeric type.

In [12]:
# --- Initial Data Cleaning ---
df = raw_df.copy()

# Drop columns with too many nulls or that are not needed
df = df.drop(columns=['energy', 'co2'])

# Drop rows with critical missing values
df = df.dropna(subset=['price', 'title', 'm2'])

# Clean original_price column
def clean_price(price):
    if pd.isna(price):
        return np.nan
    price_str = str(price)
    cleaned_price = ''.join(filter(str.isdigit, price_str))
    return int(cleaned_price) if cleaned_price else np.nan

df['original_price'] = df['original_price'].apply(clean_price)

print("Initial cleaning complete.")
df.info()

Initial cleaning complete.
<class 'pandas.core.frame.DataFrame'>
Index: 11807 entries, 0 to 11829
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              11807 non-null  int64  
 1   title           11807 non-null  object 
 2   location        11807 non-null  object 
 3   city            11807 non-null  object 
 4   price           11807 non-null  float64
 5   original_price  11807 non-null  int64  
 6   discount        11807 non-null  int64  
 7   m2              11807 non-null  float64
 8   hab             11546 non-null  float64
 9   floor           9920 non-null   object 
 10  air             11807 non-null  bool   
 11  terrace         11807 non-null  bool   
 12  storage         11807 non-null  bool   
 13  elevator        11807 non-null  bool   
 14  garage          11807 non-null  bool   
 15  outside         11807 non-null  bool   
 16  wardrobes       11807 non-null  bool   
 17  pool     

## 4. Feature Engineering

Now, we create new, more useful features from the existing raw data.

### 4.1. Engineering Features from the `floor` Column
The `floor` column contains rich but unstructured text. We will parse it to create three new, structured columns:
1.  `has_elevator`: A boolean (1/0) indicating if the property has an elevator.
2.  `property_type`: A categorical feature (`interior`, `exterior`, or `unknown`).
3.  `floor_number`: A numerical feature representing the floor level.

### 4.2. Creating `price_m2`
We will also create the `price_m2` (price per square meter) feature, a key metric for comparing property values.

In [13]:
# --- Feature Engineering from 'floor' ---

# 1. Engineer 'has_elevator'
# This was identified as more reliable than the original 'elevator' column due to scraping errors.
df['has_elevator'] = df['floor'].str.contains('con ascensor', na=False).astype(int)
df = df.drop(columns=['elevator'])

# 2. Engineer 'property_type'
def extract_property_type(floor_text):
    if pd.isna(floor_text):
        return 'unknown'
    if 'interior' in floor_text:
        return 'interior'
    if 'exterior' in floor_text:
        return 'exterior'
    return 'unknown'

df['property_type'] = df['floor'].apply(extract_property_type)

# 3. Engineer 'floor_number'
def extract_floor_number(floor_text):
    if pd.isna(floor_text):
        return np.nan
    floor_text = floor_text.lower()
    if 'bajo' in floor_text:
        return 0
    
    match = re.search(r'(-?\d+)', floor_text)
    return int(match.group(1)) if match else np.nan

df['floor_number'] = df['floor'].apply(extract_floor_number)

# 4. Drop the original 'floor' column
df = df.drop(columns=['floor'])

# --- Create 'price_m2' feature ---
df['price_m2'] = df['price'] / df['m2']

print("Feature engineering complete.")
df[['has_elevator', 'property_type', 'floor_number', 'price_m2']].head()

Feature engineering complete.


,has_elevator,property_type,floor_number,price_m2
0,1,exterior,2.0,5808.823529
1,1,exterior,1.0,5173.410405
2,0,exterior,1.0,6162.162162
3,1,interior,3.0,6466.292135
4,1,exterior,3.0,7291.666667


## 5. Handling Missing Values (Imputation)

Now we address the remaining missing values. Our strategy is:
- For categorical columns (`status`, `property_type`), we will fill missing values with the string `'unknown'`.
- For numerical columns (`floor_number`, `rooms`), we will fill missing values with the **median** of the column.

In [14]:
# --- Impute Missing Values ---

# Fill categorical NaNs
df['status'] = df['status'].fillna('unknown')
df['property_type'] = df['property_type'].fillna('unknown')

# Fill numerical NaNs with the median
floor_median = df['floor_number'].median()
df['floor_number'] = df['floor_number'].fillna(floor_median)

rooms_median = df['hab'].median()
df['hab'] = df['hab'].fillna(rooms_median)

# Rename 'hab' to 'rooms' for clarity
df = df.rename(columns={'hab': 'rooms'})

print("Missing values handled.")
print(df.isnull().sum())

Missing values handled.
id                  0
title               0
location            0
city                0
price               0
original_price      0
discount            0
m2                  0
rooms               0
air                 0
terrace             0
storage             0
garage              0
outside             0
wardrobes           0
pool                0
garden              0
status              0
satus             996
has_elevator        0
property_type       0
floor_number        0
price_m2            0
dtype: int64


## 6. Final Encoding

To prepare the data for machine learning, we need to convert all features to a numerical format.
- **Boolean columns** (`air`, `terrace`, etc.) will be converted to integers (0 or 1).
- **Categorical columns** (`status`, `property_type`) will be mapped to numerical values.

In [15]:
# --- Encode Categorical and Boolean Features ---

# Convert boolean columns to integers
boolean_columns = ['air', 'terrace', 'storage', 'garage', 'outside', 'wardrobes', 'pool', 'garden']
for col in boolean_columns:
    if col in df.columns:
        df[col] = df[col].astype(int)

# Define mappings for categorical variables
status_mapping = {
    'unknown': 0,
    'No especificado': 0,
    'Buen estado': 1,
    'A reformar': 2,
    'Nueva': 3
}

type_mapping = {
    'unknown': 0,
    'interior': 1,
    'exterior': 2
}

# Apply mappings
df['status'] = df['status'].map(status_mapping)
df['property_type'] = df['property_type'].map(type_mapping)

print("Encoding complete.")
df.head()

Encoding complete.


,id,title,location,city,price,original_price,discount,m2,rooms,air,...,outside,wardrobes,pool,garden,status,satus,has_elevator,property_type,floor_number,price_m2
0,105520205,Piso en venta en calle del Pi,El,"Gòtic,",395000.0,395000,0,68.0,2.0,1,...,1,1,0,0,0,NaN,1,2,2.0,5808.823529
1,101127926,Piso en venta en calle de la Palma de Sant Just,El,"Gòtic,",895000.0,895000,0,173.0,3.0,0,...,1,1,0,0,0,NaN,1,2,1.0,5173.410405
2,105228892,Piso en venta en El Gòtic,Ciutat,"Vella,",1140000.0,1190000,0,185.0,3.0,1,...,1,0,0,0,0,NaN,0,2,1.0,6162.162162
3,104620738,Piso en venta en calle dels Banys Nous,El,"Gòtic,",575500.0,575500,0,89.0,2.0,1,...,0,0,0,0,0,NaN,1,1,3.0,6466.292135
4,103013559,Piso en venta en El Gòtic,Ciutat,"Vella,",1400000.0,1400000,0,192.0,3.0,0,...,1,0,0,0,0,NaN,1,2,3.0,7291.666667


## 7. Final Review and Save Cleaned Data

The dataset is now clean, processed, and ready for analysis. We will save the cleaned DataFrame to a new CSV file. This file will be the input for the EDA and Modeling notebooks.

In [16]:
# --- Final Review ---
print("Final DataFrame info:")
df.info()

# --- Save the cleaned data ---
cleaned_filepath = '../data/barcelona_properties_cleaned.csv'
df.to_csv(cleaned_filepath, index=False)

print(f"\nCleaned data successfully saved to '{cleaned_filepath}'")

Final DataFrame info:
<class 'pandas.core.frame.DataFrame'>
Index: 11807 entries, 0 to 11829
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              11807 non-null  int64  
 1   title           11807 non-null  object 
 2   location        11807 non-null  object 
 3   city            11807 non-null  object 
 4   price           11807 non-null  float64
 5   original_price  11807 non-null  int64  
 6   discount        11807 non-null  int64  
 7   m2              11807 non-null  float64
 8   rooms           11807 non-null  float64
 9   air             11807 non-null  int32  
 10  terrace         11807 non-null  int32  
 11  storage         11807 non-null  int32  
 12  garage          11807 non-null  int32  
 13  outside         11807 non-null  int32  
 14  wardrobes       11807 non-null  int32  
 15  pool            11807 non-null  int32  
 16  garden          11807 non-null  int32  
 17  status        